In [1]:
import torch
from modules.data_utils import load_all_slices_from_tree

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"[Device] {device}")

[Device] cuda


In [2]:
# 1) 데이터 로드
data_root = './01_data/04_flair_preproc_slices/'
X, y = load_all_slices_from_tree(data_root, select_N=100)
print(f"[Loaded] total={len(y)} | pos={(y==1).sum()} | neg={(y==0).sum()}")

[Loaded] subjects=100, total_slices=2384, pos=22, neg=2362
[Loaded] total=2384 | pos=22 | neg=2362


In [3]:
from modules.data_utils import undersample_negatives
neg_ratio = 1.0
X_bal, y_bal = undersample_negatives(X, y, neg_ratio=neg_ratio)
print(f"[Balanced] total={len(y_bal)} | pos={(y_bal==1).sum()} | neg={(y_bal==0).sum()} (neg_ratio={neg_ratio})")

[Balanced] total=44 | pos=22 | neg=22 (neg_ratio=1.0)


In [4]:
import numpy as np
import os

from modules.data_utils import train_test_split_stratified, train_val_split_stratified
test_ratio = 0.3
val_ratio = 0.1
seed = 42
out_dir = './02_results/test'

# 3) train/test split then train/val split
(Xtr_full, ytr_full), (Xte, yte) = train_test_split_stratified(X_bal, y_bal, test_ratio=test_ratio, seed=seed)
(Xtr, ytr), (Xval, yval) = train_val_split_stratified(Xtr_full, ytr_full, val_ratio=val_ratio, seed=seed)
print(f"[Split] train={len(ytr)} | val={len(yval)} | test={len(yte)} (test_ratio={test_ratio}, val_ratio={val_ratio})")
np.save(os.path.join(out_dir, "test_data.npy"), Xte)

[Split] train=30 | val=2 | test=12 (test_ratio=0.3, val_ratio=0.1)


In [5]:
from modules.data_utils import SliceDataset

# 4) Datasets & Dataloaders
ds_train = SliceDataset(Xtr, ytr)
ds_val   = SliceDataset(Xval, yval)
ds_test  = SliceDataset(Xte, yte)

In [6]:
from torch.utils.data import DataLoader

batch_size = 16
num_workers = 4

dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True,
                        num_workers=num_workers, pin_memory=False,
                        persistent_workers=False if num_workers == 0 else True)
dl_val   = DataLoader(ds_val, batch_size=batch_size, shuffle=False,
                        num_workers=num_workers, pin_memory=False,
                        persistent_workers=False if num_workers == 0 else True)
dl_test  = DataLoader(ds_test,  batch_size=batch_size, shuffle=False,
                        num_workers=num_workers, pin_memory=False,
                        persistent_workers=False if num_workers == 0 else True)

In [23]:
len(dl_train.dataset)

30

In [7]:
for img, lab in dl_train:
    print(img.shape, lab.shape)

torch.Size([16, 1, 672, 672]) torch.Size([16])
torch.Size([14, 1, 672, 672]) torch.Size([14])


In [ ]:
from modules.model_utils import *

weight_pos = 1.0
lr = 0.003

# 5) 모델/손실/최적화
model = build_resnet18_1ch(num_classes=1).to(device)
pos_weight = torch.tensor([weight_pos], device=device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)

In [20]:
pos_weight

tensor([1.], device='cuda:0')

In [21]:
criterion

BCEWithLogitsLoss()